In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy.linalg as la
import copy

In [2]:
T = np.pi/2
tol = 10**(-9)
delta = 10**(-8)
err = 10**(-10)
x0 = 0
v0 = 0
yT = - np.pi/2
xT = 0

In [3]:
def df(funk, perem, x, y, u, v):
    if funk == 1:
        if perem ==1:
            return (vaushki(T, x, y + delta, u, v)[0] - vaushki(T, x, y, u, v)[0])/delta
        else:
            return (vaushki(T, x, y, u + delta, v)[0] - vaushki(T, x, y, u, v)[0])/delta
    if funk == 2:
        if perem == 1:
            return (vaushki(T, x, y + delta, u, v)[1] - vaushki(T, x, y, u, v)[1])/delta
        else:
            return (vaushki(T, x, y, u + delta, v)[1] - vaushki(T, x, y, u, v)[1])/delta

In [42]:
def x(t, x, y, u, v):
    return y

def y(t, x, y, u, v):
    return v - x*np.exp(-alpha*x)

def u(t, x, y, u, v):        # px
    return (1-alpha*x)*v*np.exp(-alpha*x)

def v(t, x, y, u, v):        # py
    return -u

In [13]:
def write(M):                                        # функция выводит матрицу красиво
    x = len(M)
    y = len(M[0])
    print()
    for i in range(x):
        for j in range(y):
            print("%8.3f" % M[i][j], end='')
        print()
    print('\n')

def Gauss(M, b, N):                                  # функция решает систему уравнений, заданную матрицей и вектором, методом Гаусса
    n = N + 1
    ans = [0 for i in range (N)]
    A = [[M[0][0], M[0][1], b[0]], [M[1][0], M[1][1], b[1]]]

    # прямой ход
    for s in range (N):
        diagonal = A[s][s]
        for i in range (0, n):
            A[s][i] = A[s][i]/diagonal
        for i in range (s+1, N):
            firstelement = A[i][s]
            for j in range (n):
                A[i][j] = A[i][j] - A[s][j]*firstelement

    # обратный ход
    ans[N-1] = A[N-1][N]
    for i in range (2, n):
        ans[N-i] = A[N-i][N]
        for j in range (1, i):
            ans[N-i] -= A[N-i][N-j]*ans[N-j]

def check(A, b, ans, N):
    NumpyMatrix = np.array(A)
    NumpyVector = np.array(b)
    solution = np.linalg.solve(NumpyMatrix, NumpyVector)
    print('Checking the answers:')
    for i in range (N):
        print(ans[i], "=", solution[i])              # сравниваем полученные нами результаты с результатами вычисления встроенного алгоритма решения СЛАУ

def GaussString(M, b, N):                            # функция решает систему уравнений, заданную матрицей и вектором, методом Гаусса с выбором главного элемента в столбце (меняем строки местами)
    n = N + 1
    ans = [0 for i in range (N)]
    A = [[M[0][0], M[0][1], b[0]], [M[1][0], M[1][1], b[1]]]

    for s in range (N):
        if (A[s][s] != 0):
            diagonal = A[s][s]
            for i in range (0, n):
                A[s][i] = A[s][i]/diagonal
            for i in range (s+1, N):
                firstelement = A[i][s]
                for j in range (n):
                    A[i][j] = A[i][j] - A[s][j]*firstelement
        else:
            for i in range(s+1, N):
                if (abs(A[i][s]) > abs(A[s][s])):
                    A[i], A[s] = A[s], A[i]
            diagonal = A[s][s]
            for i in range (0, n):
                A[s][i] = A[s][i]/diagonal
            for i in range (s+1, N):
                firstelement = A[i][s]
                for j in range (n):
                    A[i][j] = A[i][j] - A[s][j]*firstelement

    ans[N-1] = A[N-1][N]
    for i in range (2, n):
        ans[N-i] = A[N-i][N]
        for j in range (1, i):
            ans[N-i] -= A[N-i][N-j]*ans[N-j]

    return(ans)

In [14]:
def vaushki(T, x0, y0, u0, v0):
    
    n = 0
    
    h = 0.01
    t0 = 0
    
    fac = 0.98
    facmin = 0.7
    facmax = 1.3
    p = 5

    fx = []
    fy = []
    fu = []
    fv = []

    a21 =                           1 / 5
    a31, a32 =                      3 / 40,         9 / 40
    a41, a42, a43 =                 44 / 45,        -56 / 15,       32 / 9
    a51, a52, a53, a54 =            19372 / 6561,   -25360 / 2187,  64448 / 6561,   -212 / 729
    a61, a62, a63, a64, a65 =       9017 / 3168,    -355 / 33,      46732 / 5247,   49 / 176,   -5103 / 18656
    a71, a72, a73, a74, a75, a76 =  35 / 384,       0,              500 / 1113,     125 / 192,  -2187 / 6784,   11 / 84
    b1, b2, b3, b4, b5, b6, b7 =    35 / 384,       0,              500 / 1113,     125 / 192,  -2187 / 6784,   11 / 84,    0
    c1, c2, c3, c4, c5, c6, c7 =    0,              1 / 5,          3 / 10,         4 / 5,      8 / 9,          1,          1
    d1, d2, d3, d4, d5, d6, d7 =    5179/57600,     0,              7571/16695,     393/640,    -92097/339200,  187/2100,   1/40

    while t0 < T:
        n += 1
        
        k1x = x(t0, x0, y0, u0, v0)
        k1y = y(t0, x0, y0, u0, v0)
        k1u = u(t0, x0, y0, u0, v0)
        k1v = v(t0, x0, y0, u0, v0)
        
        k2x = x(t0 + c2 * h, x0 + h * (a21 * k1x), y0 + h * (a21 * k1y), u0 + h * (a21 * k1u), v0 + h * (a21 * k1v))
        k2y = y(t0 + c2 * h, x0 + h * (a21 * k1x), y0 + h * (a21 * k1y), u0 + h * (a21 * k1u), v0 + h * (a21 * k1v))
        k2u = u(t0 + c2 * h, x0 + h * (a21 * k1x), y0 + h * (a21 * k1y), u0 + h * (a21 * k1u), v0 + h * (a21 * k1v))
        k2v = v(t0 + c2 * h, x0 + h * (a21 * k1x), y0 + h * (a21 * k1y), u0 + h * (a21 * k1u), v0 + h * (a21 * k1v))
                
        k3x = x(t0 + c3 * h, x0 + h * (a31 * k1x + a32 * k2x), y0 + h * (a31 * k1y + a32 * k2y), u0 + h * (a31 * k1u + a32 * k2u), v0 + h * (a31 * k1v + a32 * k2v))
        k3y = y(t0 + c3 * h, x0 + h * (a31 * k1x + a32 * k2x), y0 + h * (a31 * k1y + a32 * k2y), u0 + h * (a31 * k1u + a32 * k2u), v0 + h * (a31 * k1v + a32 * k2v))
        k3u = u(t0 + c3 * h, x0 + h * (a31 * k1x + a32 * k2x), y0 + h * (a31 * k1y + a32 * k2y), u0 + h * (a31 * k1u + a32 * k2u), v0 + h * (a31 * k1v + a32 * k2v))
        k3v = v(t0 + c3 * h, x0 + h * (a31 * k1x + a32 * k2x), y0 + h * (a31 * k1y + a32 * k2y), u0 + h * (a31 * k1u + a32 * k2u), v0 + h * (a31 * k1v + a32 * k2v))
                
        k4x = x(t0 + c4 * h, x0 + h * (a41 * k1x + a42 * k2x + a43 * k3x), y0 + h * (a41 * k1y + a42 * k2y + a43 * k3y), u0 + h * (a41 * k1u + a42 * k2u + a43 * k3u), v0 + h * (a41 * k1v + a42 * k2v + a43 * k3v))
        k4y = y(t0 + c4 * h, x0 + h * (a41 * k1x + a42 * k2x + a43 * k3x), y0 + h * (a41 * k1y + a42 * k2y + a43 * k3y), u0 + h * (a41 * k1u + a42 * k2u + a43 * k3u), v0 + h * (a41 * k1v + a42 * k2v + a43 * k3v))
        k4u = u(t0 + c4 * h, x0 + h * (a41 * k1x + a42 * k2x + a43 * k3x), y0 + h * (a41 * k1y + a42 * k2y + a43 * k3y), u0 + h * (a41 * k1u + a42 * k2u + a43 * k3u), v0 + h * (a41 * k1v + a42 * k2v + a43 * k3v))
        k4v = v(t0 + c4 * h, x0 + h * (a41 * k1x + a42 * k2x + a43 * k3x), y0 + h * (a41 * k1y + a42 * k2y + a43 * k3y), u0 + h * (a41 * k1u + a42 * k2u + a43 * k3u), v0 + h * (a41 * k1v + a42 * k2v + a43 * k3v))
                
        k5x = x(t0 + c5 * h, x0 + h * (a51 * k1x + a52 * k2x + a53 * k3x + a54 * k4x), y0 + h * (a51 * k1y + a52 * k2y + a53 * k3y + a54 * k4y), u0 + h * (a51 * k1u + a52 * k2u + a53 * k3u + a54 * k4u), v0 + h * (a51 * k1v + a52 * k2v + a53 * k3v + a54 * k4v))
        k5y = y(t0 + c5 * h, x0 + h * (a51 * k1x + a52 * k2x + a53 * k3x + a54 * k4x), y0 + h * (a51 * k1y + a52 * k2y + a53 * k3y + a54 * k4y), u0 + h * (a51 * k1u + a52 * k2u + a53 * k3u + a54 * k4u), v0 + h * (a51 * k1v + a52 * k2v + a53 * k3v + a54 * k4v))
        k5u = u(t0 + c5 * h, x0 + h * (a51 * k1x + a52 * k2x + a53 * k3x + a54 * k4x), y0 + h * (a51 * k1y + a52 * k2y + a53 * k3y + a54 * k4y), u0 + h * (a51 * k1u + a52 * k2u + a53 * k3u + a54 * k4u), v0 + h * (a51 * k1v + a52 * k2v + a53 * k3v + a54 * k4v))
        k5v = v(t0 + c5 * h, x0 + h * (a51 * k1x + a52 * k2x + a53 * k3x + a54 * k4x), y0 + h * (a51 * k1y + a52 * k2y + a53 * k3y + a54 * k4y), u0 + h * (a51 * k1u + a52 * k2u + a53 * k3u + a54 * k4u), v0 + h * (a51 * k1v + a52 * k2v + a53 * k3v + a54 * k4v))
        
        k6x = x(t0 + c6 * h, x0 + h * (a61 * k1x + a62 * k2x + a63 * k3x + a64 * k4x + a65 * k5x), y0 + h * (a61 * k1y + a62 * k2y + a63 * k3y + a64 * k4y + a65 * k5y), u0 + h * (a61 * k1u + a62 * k2u + a63 * k3u + a64 * k4u + a65 * k5u), v0 + h * (a61 * k1v + a62 * k2v + a63 * k3v + a64 * k4v + a65 * k5v))
        k6y = y(t0 + c6 * h, x0 + h * (a61 * k1x + a62 * k2x + a63 * k3x + a64 * k4x + a65 * k5x), y0 + h * (a61 * k1y + a62 * k2y + a63 * k3y + a64 * k4y + a65 * k5y), u0 + h * (a61 * k1u + a62 * k2u + a63 * k3u + a64 * k4u + a65 * k5u), v0 + h * (a61 * k1v + a62 * k2v + a63 * k3v + a64 * k4v + a65 * k5v))
        k6u = u(t0 + c6 * h, x0 + h * (a61 * k1x + a62 * k2x + a63 * k3x + a64 * k4x + a65 * k5x), y0 + h * (a61 * k1y + a62 * k2y + a63 * k3y + a64 * k4y + a65 * k5y), u0 + h * (a61 * k1u + a62 * k2u + a63 * k3u + a64 * k4u + a65 * k5u), v0 + h * (a61 * k1v + a62 * k2v + a63 * k3v + a64 * k4v + a65 * k5v))
        k6v = v(t0 + c6 * h, x0 + h * (a61 * k1x + a62 * k2x + a63 * k3x + a64 * k4x + a65 * k5x), y0 + h * (a61 * k1y + a62 * k2y + a63 * k3y + a64 * k4y + a65 * k5y), u0 + h * (a61 * k1u + a62 * k2u + a63 * k3u + a64 * k4u + a65 * k5u), v0 + h * (a61 * k1v + a62 * k2v + a63 * k3v + a64 * k4v + a65 * k5v))
        
        k7x = x(t0 + c7 * h, x0 + h * (a71 * k1x + a72 * k2x + a73 * k3x + a74 * k4x + a75 * k5x + a76 * k6x), y0 + h * (a71 * k1y + a72 * k2y + a73 * k3y + a74 * k4y + a75 * k5y + a76 * k6y), u0 + h * (a71 * k1u + a72 * k2u + a73 * k3u + a74 * k4u + a75 * k5u + a76 * k6u), v0 + h * (a71 * k1v + a72 * k2v + a73 * k3v + a74 * k4v + a75 * k5v + a76 * k6v))
        k7y = y(t0 + c7 * h, x0 + h * (a71 * k1x + a72 * k2x + a73 * k3x + a74 * k4x + a75 * k5x + a76 * k6x), y0 + h * (a71 * k1y + a72 * k2y + a73 * k3y + a74 * k4y + a75 * k5y + a76 * k6y), u0 + h * (a71 * k1u + a72 * k2u + a73 * k3u + a74 * k4u + a75 * k5u + a76 * k6u), v0 + h * (a71 * k1v + a72 * k2v + a73 * k3v + a74 * k4v + a75 * k5v + a76 * k6v))
        k7u = u(t0 + c7 * h, x0 + h * (a71 * k1x + a72 * k2x + a73 * k3x + a74 * k4x + a75 * k5x + a76 * k6x), y0 + h * (a71 * k1y + a72 * k2y + a73 * k3y + a74 * k4y + a75 * k5y + a76 * k6y), u0 + h * (a71 * k1u + a72 * k2u + a73 * k3u + a74 * k4u + a75 * k5u + a76 * k6u), v0 + h * (a71 * k1v + a72 * k2v + a73 * k3v + a74 * k4v + a75 * k5v + a76 * k6v))
        k7v = v(t0 + c7 * h, x0 + h * (a71 * k1x + a72 * k2x + a73 * k3x + a74 * k4x + a75 * k5x + a76 * k6x), y0 + h * (a71 * k1y + a72 * k2y + a73 * k3y + a74 * k4y + a75 * k5y + a76 * k6y), u0 + h * (a71 * k1u + a72 * k2u + a73 * k3u + a74 * k4u + a75 * k5u + a76 * k6u), v0 + h * (a71 * k1v + a72 * k2v + a73 * k3v + a74 * k4v + a75 * k5v + a76 * k6v))
        
        
        x1 = x0 + (h * (k1x * b1 + k2x * b2 + k3x * b3 + k4x * b4 + k5x * b5 + k6x * b6 + k7x * b7))
        y1 = y0 + (h * (k1y * b1 + k2y * b2 + k3y * b3 + k4y * b4 + k5y * b5 + k6y * b6 + k7y * b7))
        u1 = u0 + (h * (k1u * b1 + k2u * b2 + k3u * b3 + k4u * b4 + k5u * b5 + k6u * b6 + k7u * b7))
        v1 = v0 + (h * (k1v * b1 + k2v * b2 + k3v * b3 + k4v * b4 + k5v * b5 + k6v * b6 + k7v * b7))
        
        x1_shtrix = x0 + (h * (k1x * d1 + k2x * d2 + k3x * d3 + k4x * d4 + k5x * d5 + k6x * d6 + k7x * d7))
        y1_shtrix = y0 + (h * (k1y * d1 + k2y * d2 + k3y * d3 + k4y * d4 + k5y * d5 + k6y * d6 + k7y * d7))
        u1_shtrix = u0 + (h * (k1u * d1 + k2u * d2 + k3u * d3 + k4u * d4 + k5u * d5 + k6u * d6 + k7u * d7))
        v1_shtrix = v0 + (h * (k1v * d1 + k2v * d2 + k3v * d3 + k4v * d4 + k5v * d5 + k6v * d6 + k7v * d7))
        
        x_err = abs(x1 - x1_shtrix)
        y_err = abs(y1 - y1_shtrix)
        u_err = abs(u1 - u1_shtrix)
        v_err = abs(v1 - v1_shtrix)
        
        
        err = max(x_err, y_err, u_err, v_err)
        if (err <= tol):
            x0 = x1
            y0 = y1
            u0 = u1
            v0 = v1
            
            t0 += h
            h = h * min(facmax, max(facmin, fac*((tol/err)**(1/(p+1)))))
        else:
            h = h * min(facmax, max(facmin, fac*((tol/err)**(1/(p+1)))))
        if (t0 + h > T):
            h = T - t0

        fx.append(x0)
        fy.append(y0)
        fu.append(u0)
        fv.append(v0)
    
    return(fx[-1], fy[-1], fu[-1], fv[-1])

In [43]:
for alpha in [0, 0.5, 1, 5, 10, 25]:
    y0 = 3
    u0 = -7

    alpha0 = np.array([y0, u0])
    X = np.array([xT - vaushki(T, x0, alpha0[0], alpha0[1], v0)[0], yT - vaushki(T, x0, alpha0[0], alpha0[1], v0)[1]])
    count = 0
    
    print('α =', alpha)
    print('start from', alpha0)
    while np.sqrt(X[0]**2 + X[1]**2) > err:
        count += 1
        if count > 100:
            print('too many iterations')
            break

        n = 0
        X = np.array([xT - vaushki(T, x0, alpha0[0], alpha0[1], v0)[0], yT - vaushki(T, x0, alpha0[0], alpha0[1], v0)[1]])
        A = np.array([[df(1, 1, x0, alpha0[0], alpha0[1], v0), df(1, 2, x0, alpha0[0], alpha0[1], v0)], 
                      [df(2, 1, x0, alpha0[0], alpha0[1], v0), df(2, 2, x0, alpha0[0], alpha0[1], v0)]])
        A_inv = np.linalg.inv(A)
        h = np.array([A_inv[0][0]*X[0] + A_inv[0][1]*X[1], A_inv[1][0]*X[0] + A_inv[1][1]*X[1]])
        
        alpha1 = np.array([alpha0[0] + h[0], alpha0[1] + h[1]])

        new_X = np.array([xT - vaushki(T, x0, alpha1[0], alpha1[1], v0)[0], yT - vaushki(T, x0, alpha1[0], alpha1[1], v0)[1]])

        if (np.sqrt(new_X[0]**2 + new_X[1]**2) < np.sqrt(X[0]**2 + X[1]**2)):
            alpha0 = alpha1
        else:
            while (np.sqrt(new_X[0]**2 + new_X[1]**2) >= np.sqrt(X[0]**2 + X[1]**2)):
                n -= 1
                if n < -30:
                    print(count, 'too large power')
                    break
                alpha1 = alpha0 + h*(2**n)
                new_X = np.array([xT - vaushki(T, x0, alpha1[0], alpha1[1], v0)[0], yT - vaushki(T, x0, alpha1[0], alpha1[1], v0)[1]])
            alpha0 = alpha1
    
    print('Количество итераций:', count)
    print('Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно', alpha0[0])
    print('Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно', alpha0[1])
    print('x(T) =', new_X[0])
    print('y(T) = - π/2 +', new_X[1])
    print()

α = 0
start from [ 3 -7]
Количество итераций: 3
Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно 1.0000000005885634
Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно 2.0000000001040084
x(T) = -7.407269242420966e-16
y(T) = - π/2 + -1.3322676295501878e-15

α = 0.5
start from [ 3 -7]
Количество итераций: 5
Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно 0.936925268716248
Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно 1.9215839777703496
x(T) = 8.049116928532385e-16
y(T) = - π/2 + 1.5543122344752192e-15

α = 1
start from [ 3 -7]
Количество итераций: 5
Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно 0.8956807432861554
Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно 1.8739938257350672
x(T) = -1.2836953722228372e-16
y(T) = - π/2 + 0.0

α = 5
start from [ 3 -7]


C:\Users\pfonk\AppData\Local\Temp/ipykernel_8964/1121895731.py:91: RuntimeWarning: divide by zero encountered in double_scalars
  h = h * min(facmax, max(facmin, fac*((tol/err)**(1/(p+1)))))


Количество итераций: 5
Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно 0.796751904718245
Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно 1.8201672797750503
x(T) = 3.400058012914542e-16
y(T) = - π/2 + 4.440892098500626e-16

α = 10
start from [ 3 -7]
Количество итераций: 5
Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно 0.7874602990437711
Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно 1.8697287406945193
x(T) = -5.551115123125783e-17
y(T) = - π/2 + 4.440892098500626e-16

α = 25
start from [ 3 -7]
Количество итераций: 4
Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно 0.7863569367028215
Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно 1.9038449841262863
x(T) = 9.566999970012091e-16
y(T) = - π/2 + 1.9984014443252818e-15



In [44]:
for alpha in [0, 0.5, 1, 5, 10, 25]:
    y0 = 3
    u0 = -7

    alpha0 = np.array([y0, u0])
    X = np.array([xT - vaushki(T, x0, alpha0[0], alpha0[1], v0)[0], yT - vaushki(T, x0, alpha0[0], alpha0[1], v0)[1]])
    count = 0
    
    print('α =', alpha)
    print('start from', alpha0)
    while np.sqrt(X[0]**2 + X[1]**2) > err:
        count += 1
        if count > 100:
            print('too many iterations')
            break

        n = 0
        X = np.array([xT - vaushki(T, x0, alpha0[0], alpha0[1], v0)[0], yT - vaushki(T, x0, alpha0[0], alpha0[1], v0)[1]])
        A = np.array([[df(1, 1, x0, alpha0[0], alpha0[1], v0), df(1, 2, x0, alpha0[0], alpha0[1], v0)], 
                      [df(2, 1, x0, alpha0[0], alpha0[1], v0), df(2, 2, x0, alpha0[0], alpha0[1], v0)]])
        A_inv = np.linalg.inv(A)
        
        N = len(A)                                               # размерноесть матрицы и длина вектора
        Matrix = A
        Vector = X
        np.resize(X, [1, N])

        h = GaussString(A, X, N)
        
        alpha1 = np.array([alpha0[0] + h[0], alpha0[1] + h[1]])

        new_X = np.array([xT - vaushki(T, x0, alpha1[0], alpha1[1], v0)[0], yT - vaushki(T, x0, alpha1[0], alpha1[1], v0)[1]])

        if (np.sqrt(new_X[0]**2 + new_X[1]**2) < np.sqrt(X[0]**2 + X[1]**2)):
            alpha0 = alpha1
        else:
            while (np.sqrt(new_X[0]**2 + new_X[1]**2) >= np.sqrt(X[0]**2 + X[1]**2)):
                n -= 1
                if n < -30:
                    print(count, 'too large power')
                    break
                alpha1 = np.array([alpha0[0] + h[0]*(2**n), alpha0[1] + h[1]*(2**n)])
                #alpha1 = alpha0 + h*(2**n)
                new_X = np.array([xT - vaushki(T, x0, alpha1[0], alpha1[1], v0)[0], yT - vaushki(T, x0, alpha1[0], alpha1[1], v0)[1]])
            alpha0 = alpha1
    
    print('Количество итераций:', count)
    print('Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно', alpha0[0])
    print('Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно', alpha0[1])
    print('x(T) =', new_X[0])
    print('y(T) = - π/2 +', new_X[1])
    print()

α = 0
start from [ 3 -7]
Количество итераций: 3
Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно 1.0000000005885634
Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно 2.0000000001040097
x(T) = 4.354155924701786e-16
y(T) = - π/2 + 2.220446049250313e-16

α = 0.5
start from [ 3 -7]
Количество итераций: 5
Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно 0.9369252687162478
Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно 1.9215839777703474
x(T) = -4.0245584642661925e-16
y(T) = - π/2 + -8.881784197001252e-16

α = 1
start from [ 3 -7]
Количество итераций: 5
Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно 0.8956807432861552
Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно 1.8739938257350668
x(T) = -2.42861286636753e-17
y(T) = - π/2 + 2.220446049250313e-16

α = 5
start from [ 3 -7]


C:\Users\pfonk\AppData\Local\Temp/ipykernel_8964/1121895731.py:91: RuntimeWarning: divide by zero encountered in double_scalars
  h = h * min(facmax, max(facmin, fac*((tol/err)**(1/(p+1)))))


Количество итераций: 5
Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно 0.7967519047182451
Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно 1.8201672797750497
x(T) = -9.020562075079397e-16
y(T) = - π/2 + -1.3322676295501878e-15

α = 10
start from [ 3 -7]
Количество итераций: 5
Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно 0.7874602990437709
Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно 1.8697287406945187
x(T) = 1.3357370765021415e-15
y(T) = - π/2 + 2.220446049250313e-15

α = 25
start from [ 3 -7]
Количество итераций: 4
Значение функции y в точке 0, при котором условия краевой задачи выполняются, равно 0.7863569367028214
Значение функции px в точке 0, при котором условия краевой задачи выполняются, равно 1.9038449841262859
x(T) = 7.398595625041082e-16
y(T) = - π/2 + 8.881784197001252e-16

